In [67]:
from geojson import FeatureCollection, Feature
import http.client
import pandas as pd
import numpy as np
import requests
import json
import geopandas as gpd
from datetime import datetime
import io

In [237]:
anoka_input_dir = r"Z:\Data\Users\Sijia\Met_Council\GIS\CountyLanes_3-27-19\Anoka\AnokaCo_Segments_032519.shp"
hennepin_input_dir = r"Z:\Data\Users\Sijia\Met_Council\GIS\CountyLanes_3-27-19\Hennepin\Roadway_Width_032619.shp"
carver_input_dir = r"Z:\Data\Users\Sijia\Met_Council\GIS\CountyLanes_3-27-19\Carver County\Carver_export.shp"
dakota_input_dir = r"Z:\Data\Users\Sijia\Met_Council\GIS\CountyLanes_3-27-19\Dakota\Dakota_export.shp"

In [348]:
def shp_to_json(input_gdf):
    input_gdf_latlon = input_gdf.copy()
    input_gdf_latlon = input_gdf_latlon[input_gdf_latlon.geometry.notnull()]
    
    input_gdf_latlon['geometry'] = input_gdf_latlon['geometry'].apply(lambda g: LineString([xy[0:2] for xy in list(g.coords)]))
    
    input_gdf_latlon = input_gdf_latlon.to_crs({'init': 'epsg:4326'})
    
    obj_list = [x for x in input_gdf_latlon.columns.values if (input_gdf_latlon[x].dtype == object)&(x!='geometry')]
    for x in obj_list:
        input_gdf_latlon[x] = input_gdf_latlon[x].str.replace('\'', '')
        input_gdf_latlon[x] = input_gdf_latlon[x].str.replace('\"', '')
    
    input_gdf_latlon.fillna(0, inplace = True)
        
    input_json = json.loads(input_gdf_latlon.to_json())
    
    return input_json



def match_geom(input_json, laneKey, group_size):
    input_data = input_json['features']
    subset = (np.arange(0, len(input_data), group_size)).tolist() + [len(input_data)]
    
    unmatched_df = pd.DataFrame()
    matched_df = pd.DataFrame()
    
    for i in range(len(subset)-1):
        input_subset = input_data[subset[i] : subset[i+1]]
        
        try:
            conn = http.client.HTTPSConnection("api.sharedstreets.io")    
            fc = {"type":"FeatureCollection", "features":input_subset}
            payload = str(fc).replace('\'', '\"')
    
            headers = {"content-type": "application/json"}

            conn.request("POST", 
                     "/v0.1.0/match/geoms?authKey=bdd23fa1-7ac5-4158-b354-22ec946bb575", 
                     payload, 
                     headers)

            res = conn.getresponse()
            return_data = res.read()
    
            return_json = json.loads(return_data.decode('utf-8'))
    
            if return_json['unmatched'] is not None:
                unmatched_df = pd.concat([unmatched_df,
                                     pd.DataFrame(return_json['unmatched']['features'])],
                                     sort = False,
                                     ignore_index = True)
    
            # matched database
            features = return_json['matched']['features']
            rows = []
            for feature in features:
                row = {'referenceId' : feature['properties']['referenceId'],
                  'fromIntersectionId' : feature['properties']['fromIntersectionId'],
                  'toIntersectionId' : feature['properties']['toIntersectionId'],
                  'roadClass' : feature['properties']['roadClass'],
                  'direction' : feature['properties']['direction'],
                  'referenceLength' : feature['properties']['referenceLength'],
                  'side' : feature['properties']['side'],
                  'score' : feature['properties'].get('score', None),
                  'originalFeatureId' : feature['properties']['originalFeature']['properties']['LINK_ID'],
                  'originalFeatureLanes' : feature['properties']['originalFeature']['properties'][laneKey]}
            
                rows.append(row)
    
            matched_df = pd.concat([matched_df, 
                               pd.DataFrame(rows)],
                               sort = False,
                               ignore_index = True)
        
        except:
            print((subset[i], subset[i+1]))
            continue
    
    unmatched_df['originalFeatureId'] = unmatched_df.properties.apply(lambda x: x['LINK_ID'])
    
    return matched_df, unmatched_df



def main(input_path, matched_dir, unmatched_dir, laneKey, group_size = 200):
    print('start: ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    input_gdf = gpd.read_file(input_path)[[laneKey, 'geometry']]
    input_gdf['LINK_ID'] = range(1, len(input_gdf) + 1)
    input_json = shp_to_json(input_gdf)
    matched_df, unmatched_df = match_geom(input_json, laneKey, group_size)
    
    print('end: ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '\n')
    
    matched_df.to_csv(matched_dir, index = False)
    unmatched_df.to_csv(unmatched_dir, index = False)
    
    return matched_df, unmatched_df


def retry_match(input_path, previous_matched, previous_unmatched, laneKey, group_size):
    previous_matched_df = pd.read_csv(previous_matched)
    previous_unmatched_df = pd.read_csv(previous_unmatched)
    input_gdf = gpd.read_file(input_path)[[laneKey, 'geometry']]
    
    input_gdf['LINK_ID'] = range(1, len(input_gdf) + 1)
    input_gdf = input_gdf[-input_gdf['LINK_ID'].isin(previous_matched_df.originalFeatureId.unique().tolist())]
    input_gdf = input_gdf[-input_gdf['LINK_ID'].isin(previous_unmatched_df.originalFeatureId.unique().tolist())]

    input_json = shp_to_json(input_gdf)
    
    print('start: ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    matched_df, unmatched_df = match_geom(input_json, laneKey, group_size)
    
    print('end: ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
    matched_df = pd.concat([previous_matched_df, matched_df], sort = False, ignore_index = True)
    unmatched_df = pd.concat([previous_unmatched_df, unmatched_df], sort = False, ignore_index = True)

    matched_df.to_csv(previous_matched, index = False)
    unmatched_df.to_csv(previous_unmatched, index = False)
    
    return matched_df, unmatched_df

In [331]:
matched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\matched\matched_anoka.csv"
unmatched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\unmatched\unmatched_anoka.csv"
anoka_matched, anoka_unmatched = main(anoka_input_dir, matched_dir, unmatched_dir, 'LANES', 200)
#input_gdf = gpd.read_file(anoka_input_dir)
#input_json = shp_to_json(input_gdf)
#matched_df, unmatched_df = match_geom(input_json, 'LANES')

start: 2019-04-13 15:09:30
end: 2019-04-13 15:09:51



In [332]:
matched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\matched\matched_hennepin.csv"
unmatched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\unmatched\unmatched_hennepin.csv"
hennepin_matched, hennepin_unmatched = main(hennepin_input_dir, matched_dir, unmatched_dir, 'Lanes', 200)

start: 2019-04-13 15:10:01
end: 2019-04-13 15:11:09



In [365]:
matched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\matched\matched_carver.csv"
unmatched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\unmatched\unmatched_carver.csv"
carver_matched, carver_unmatched = main(carver_input_dir, matched_dir, unmatched_dir, 'Lanes', 1)

start: 2019-04-13 23:12:05
(18, 19)
(79, 80)
end: 2019-04-13 23:14:12



In [333]:
matched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\matched\matched_dakota.csv"
unmatched_dir = r"Z:\Data\Users\Sijia\Met_Council\osm_roadway_network_builder\lane_conflation\unmatched\unmatched_dakota.csv"
dakota_matched, dakota_unmatched = main(dakota_input_dir, matched_dir, unmatched_dir, 'No_of_Traf', 200)
#input_gdf = gpd.read_file(dakota_input_dir)
#input_gdf['geometry'] = input_gdf['geometry'].apply(lambda g: LineString([xy[0:2] for xy in list(g.coords)]))
#input_gdf = input_gdf.to_crs({'init': 'epsg:4326'})
#list(input_gdf.geometry[100].coords)
#input_gdf.crs
#input_gdf = input_gdf[input_gdf.geometry.notnull()]
#input_gdf = input_gdf.to_crs({'init': 'epsg:4326'})

start: 2019-04-13 15:11:30
end: 2019-04-13 15:11:47

